In [1]:
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
DATA_PATH = '/home/azureuser/hack/data'
OFFERS_FILE = '20210521_offers.csv'
CHECKS_FILE = '20210518_checks.csv'
HIERARHY_FILE = '20210518_hierarchy.csv'
UPLIFT_FILE = '20210518_uplift.csv'

offers_df = pd.read_csv(os.path.join(DATA_PATH, OFFERS_FILE))
checks_df = pd.read_csv(os.path.join(DATA_PATH, CHECKS_FILE))
hierarhy_df = pd.read_csv(os.path.join(DATA_PATH, HIERARHY_FILE))
uplift_df = pd.read_csv(os.path.join(DATA_PATH, UPLIFT_FILE))

In [3]:
checks_df['day'] = pd.to_datetime(checks_df['day'], format='%Y%m%d')

In [4]:
for col in ['start_date', 'end_date']:
    offers_df[col] = pd.to_datetime(offers_df[col], format='%Y%m%d')

# Simple descision

In [5]:
merged_df = offers_df.merge(uplift_df, on='Offer_ID')
merged_df.head()

,Promo_type,Offer_ID,sku,start_date,end_date,train_test_group,UpLift
0,Facades,F26,71c770923102af052f64c6036c9dd590,2020-02-13,2020-02-19,train,0.430811
1,Facades,F26,b178ad83d2ed46e4ca3b7d6bf3e50baa,2020-02-13,2020-02-19,train,0.430811
2,Facades,F26,bc169501599f610c529db4962942d80d,2020-02-13,2020-02-19,train,0.430811
3,Facades,F26,bc5f3166f87117e513a68ba6bad1d2da,2020-02-13,2020-02-19,train,0.430811
4,Facades,F26,aa4dfbef8de268ea041b073c0b06c460,2020-02-13,2020-02-19,train,0.430811


Аггрегация данных по типу промо-акции и sku

In [6]:
pred_promo = merged_df[['Promo_type', 
                        'Offer_ID', 
                        'sku', 
                        'UpLift']].drop_duplicates().groupby('Promo_type').mean()

pred_sku =  merged_df[['Promo_type', 
                       'Offer_ID', 
                       'sku', 
                       'UpLift']].drop_duplicates().groupby('sku').mean()

Датасет, для которого необходимо сделать прогноз:

In [7]:
offers_test = offers_df[offers_df['train_test_group'] == 'test']

По ключам 'Promo_type'-'sku', 'Promo_type', 'sku' объединяем датасеты с усреднёнными по трейну значениями UpLift с тестовым датасетом. Получаем прогноз Uplift по этим ключам.

In [8]:
offers_test1 = offers_test.merge(
    merged_df[['Promo_type', 'sku', 'UpLift']], 
    on=['Promo_type', 'sku'], 
    how='left').groupby(['Offer_ID', 'Promo_type', 'sku']).mean()

In [9]:
offers_test1.reset_index().head()

,Offer_ID,Promo_type,sku,UpLift
0,100,Seasonal,12e3ac7c9bf202d41ec92d747d92a234,NaN
1,100,Seasonal,228f88c28e3b259103f548e4f7788904,0.785098
2,100,Seasonal,3e0fb22a7c69510b08869627f6214003,0.785098
3,100,Seasonal,4d1581a093a0eb7751e1e0f62dcbe278,0.785098
4,100,Seasonal,752e96b000091940545ce06692bf67c5,0.785098


In [10]:
pred_promo.reset_index().head()

,Promo_type,UpLift
0,Billboards,3.620010
1,Biweekly,5.561029
2,Facades,2.630749
3,Seasonal,3.708450


In [11]:
offers_test2 = offers_test1.reset_index().merge(pred_promo, on=['Promo_type'])

In [12]:
offers_test3 = offers_test2.merge(
    pred_sku.rename(columns={'UpLift': 'UpLift_sku'}),
    on='sku',
    how='left')

Рассчитываем геометрическое среднее UpLift по полученным данным

In [13]:
offers_test3['UpLift'] = \
np.sqrt(offers_test3['UpLift_y'].fillna(1) * offers_test3['UpLift_sku'].fillna(1)).fillna(0)

/anaconda/envs/py38_default/lib/python3.8/site-packages/pandas/core/arraylike.py:358: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


Аггрегируем результаты по Offer_ID

In [14]:
test_results = offers_test3.groupby('Offer_ID').agg({'UpLift': 'mean'})

Готовим сабмит

In [15]:
sample_submission = pd.read_csv("20210521_sample_submission.csv")

In [16]:
out = sample_submission.drop(columns='UpLift').merge(test_results, on='Offer_ID')
out.head()

,Offer_ID,UpLift
0,386,8.761159
1,182,4.719755
2,173,5.684745
3,375,4.910266
4,379,8.606629


In [17]:
out.to_csv('submit_simple_decision.csv', index=False)